In [1]:
!pip install vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
  Attempting uninstall: numba
    Found existing installation: numba 0.58.1
    Uninstalling numba-0.58.1:
      Succes

In [1]:
import pandas as pd
import numpy as np
import datetime
import vectorbt as vbt

In [ ]:
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)

btc_price = vbt.YFData.download(
    "BTC-USD",
    missing_index = 'drop',
    interval = '1D'
).get("Close")

print(btc_price)

Date
2014-09-17 00:00:00+00:00      457.334015
2014-09-18 00:00:00+00:00      424.440002
2014-09-19 00:00:00+00:00      394.795990
2014-09-20 00:00:00+00:00      408.903992
2014-09-21 00:00:00+00:00      398.821014
                                 ...     
2024-07-11 00:00:00+00:00    57344.914062
2024-07-12 00:00:00+00:00    57899.464844
2024-07-13 00:00:00+00:00    59231.953125
2024-07-14 00:00:00+00:00    60787.792969
2024-07-15 00:00:00+00:00    62845.628906
Freq: D, Name: Close, Length: 3590, dtype: float64


## Easy case

In [ ]:
def ma_strategy(close, window = 730, lower_multiplier=1, upper_multiplier=4):
  signal = np.full(close.shape, np.nan)
  for x in range(len(close)):
    if x >= window:
      mavg = np.mean( close[x-window:x])
      if close[x] < mavg*lower_multiplier:
        signal[x] = 1
      elif close[x] > mavg*upper_multiplier:
        signal[x] = -1
      else:
        signal[x] = 0
  return signal


In [ ]:

ind = vbt.IndicatorFactory(
    class_name = "ma_strategy",
    short_name = "ma",
    input_names = ["close"],
    param_names = ["window", "lower_multiplier", "upper_multiplier"],
    output_names = ["signal"]).from_apply_func(
        ma_strategy,
        window = 730,
        lower_multiplier = 1,
        upper_multiplier = 4)



In [ ]:
res = ind.run(btc_price)
print(res.signal)

Date
2014-09-17 00:00:00+00:00    NaN
2014-09-18 00:00:00+00:00    NaN
2014-09-19 00:00:00+00:00    NaN
2014-09-20 00:00:00+00:00    NaN
2014-09-21 00:00:00+00:00    NaN
                            ... 
2024-07-11 00:00:00+00:00    0.0
2024-07-12 00:00:00+00:00    0.0
2024-07-13 00:00:00+00:00    0.0
2024-07-14 00:00:00+00:00    0.0
2024-07-15 00:00:00+00:00    0.0
Freq: D, Name: Close, Length: 3590, dtype: float64


In [ ]:
entris = res.signal == 1.0
exits = res.signal == -1.0

pf = vbt.Portfolio.from_signals(btc_price, entris, exits)
print(pf.stats())

Start                         2014-09-17 00:00:00+00:00
End                           2024-07-15 00:00:00+00:00
Period                               3590 days 00:00:00
Start Value                                       100.0
End Value                                   1302.953801
Total Return [%]                            1202.953801
Benchmark Return [%]                       13641.735113
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               61.81085
Max Drawdown Duration                 534 days 00:00:00
Total Trades                                          2
Total Closed Trades                                   1
Total Open Trades                                     1
Open Trade PnL                               596.816391
Win Rate [%]                                      100.0
Best Trade [%]                                606.13741
Worst Trade [%]                               60

In [ ]:
pf.total_return()

12.029538010892052

## Dashbording

In [ ]:
vbt.settings.set_theme('light')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 500

In [ ]:
pf.plot().show()

In [ ]:
# individually
pf.trades.plot_pnl().show()

In [ ]:
pf.trades.plot().show()

In [ ]:
pf.orders.plot().show()

In [ ]:
print(pf.orders.records_arr)

In [ ]:
fast_ma = vbt.MA.run(btc_price, window=50)
slow_ma = vbt.MA.run(btc_price, window=200)

entries = fast_ma.ma_crossed_above(slow_ma)
exits = fast_ma.ma_crossed_below(slow_ma)

In [ ]:

fig = btc_price.vbt.plot(trace_kwargs=dict(name='Price', line=dict(color='black')))
fig = fast_ma.ma.vbt.plot(trace_kwargs=dict(name='fast_ma', line=dict(color='blue')), fig=fig)
fig = slow_ma.ma.vbt.plot(trace_kwargs=dict(name='slow_ma', line=dict(color='green')), fig=fig)
fig = entries.vbt.signals.plot_as_entry_markers(btc_price, fig=fig)
fig = exits.vbt.signals.plot_as_exit_markers(btc_price, fig=fig)
fig.show()

In [ ]:
pf =vbt.Portfolio.from_signals(btc_price, entris, exits)
fig = pf.plot(subplots=[('price', dict(title='Price', yaxis_kwargs=dict(title='Price'))),'orders', 'trade_pnl', 'cum_returns', 'drawdowns'])
scatter = vbt.plotting.Scatter(
    data = btc_price,
    x_labels = btc_price.index,
    trace_names = ['Price'],
    trace_kwargs=dict(line=dict(color='red')),
    add_trace_kwargs = dict(row=1, col=1),
    fig = fig)

fast_ma_scatter = vbt.plotting.Scatter(
    data = fast_ma.ma,
    x_labels = fast_ma.ma.index,
    trace_names = ['fast_ma'],
    trace_kwargs=dict(line=dict(color='green')),
    add_trace_kwargs = dict(row=1, col=1),
    fig = fig)

slow_ma_scatter = vbt.plotting.Scatter(
    data = slow_ma.ma,
    x_labels = slow_ma.ma.index,
    trace_names = ['slow_ma'],
    trace_kwargs=dict(line=dict(color='blue')),
    add_trace_kwargs = dict(row=1, col=1),
    fig = fig)

fig.add_hline(y=38000, line_color='black', row=1, col=1, line_width=2)

fig.show()

## Custom indicators

In [ ]:
# get data

end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)

btc_price = vbt.YFData.download(
    ["BTC-USD", "ETH-USD"],
    missing_index = 'drop',
    end = end_time,
    start = start_time,
    interval = "1m"
).get("Close")

# def the indicators

def custom_indicator(close, rsi_window=14, ma_window=50, entry=30, exit=70):
  close_5m = close.resample("5T").last()
  rsi = vbt.RSI.run(close_5m, window=rsi_window).rsi
  rsi, _ = rsi.align(close, broadcast_axis=0, method='ffill', join='right')

  close = close.to_numpy()
  rsi = rsi.to_numpy()

  ma = vbt.MA.run(close, window=ma_window).ma.to_numpy()
  signal = np.where(rsi > exit, -1, 0)
  signal = np.where((rsi < entry) & (close < ma), 1, signal)

  return signal


# my own indicator
my_indicator = vbt.IndicatorFactory(
    class_name = "CostumIndicator",
    short_name = "Custom",
    input_names = ['close'],
    param_names = ['rsi_window', 'ma_window', 'entry', 'exit'],
    output_names = ['signal']
    ).from_apply_func(custom_indicator, rsi_window=14, ma_window=50, entry=30, exit=70, keep_pd=True)

# get the result
result = my_indicator.run(btc_price, rsi_window=np.arange(10, 40, step=3, dtype=int),
                          #ma_window=np.arange(20, 200, step=20, dtype=int),
                          entry=np.arange(10, 40, step=4, dtype=float),
                          exit=np.arange(60, 85, step=4, dtype=float),
                          param_product=True)
print(result.signal.to_string())

# get the entris and exits
entris = result.signal == 1.0
exits = result.signal == -1.0

# run the portfolio
pf = vbt.Portfolio.from_signals(btc_price, entris, exits, sl_stop=0.005, sl_trail=True,
                                upon_stop_exit=vbt.portfolio.enums.StopExitMode.Reverse,, tp_stop=0.001,freq='1m')
# print(pf.stats())
returns = pf.total_return()

# If you want to optimize each pair
# returns = returns[returns.index.isin(['BTC-USD'], level='symbol')]

# returns = returns.groupby(level=['Custom_exit', 'Custom_entry', 'symbol']).mean()

print(returns.to_string())

print(returns.max())
print(returns.idxmax())

# If you want to plot (only for one pair)
# pf.plot().show()

In [ ]:
# heatmap 2D
#fig = returns.vbt.heatmap(x_level='Custom_exit', y_level='Custom_entry', slider_level='symbol')
#fig.show()

In [ ]:
# volume
fig = returns.vbt.volume(x_level='Custom_rsi_window', y_level='Custom_entry', z_level='Custom_exit', slider_level='symbol')
fig.show()

## OPtimization Tech

In [ ]:
# get data
from numba import njit

end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(days=2)

btc_price = vbt.YFData.download(
    ["BTC-USD", "ETH-USD"],
    missing_index = 'drop',
    end = end_time,
    start = start_time,
    interval = "1m"
).get("Close")

#btc_price.to_csv("data.csv")


#btc_price = pd.read_csv('data.csv')
#btc_price['Datetime'] = pd.to_datetime(btc_price['Datetime'])
#btc_price.set_index('Datetime', inplace=True)

print(btc_price)

# def the indicators

@njit
def produce_signal(rsi, exit, entry):
  signal = np.where(rsi > exit, -1, 0)
  signal = np.where((rsi < entry), 1, signal)

  return signal

def custom_indicator(close, rsi_window=14, entry=30, exit=70):
  rsi = vbt.RSI.run(close, window=rsi_window).rsi.to_numpy()
  return produce_signal(rsi, exit, entry)



# my own indicator
my_indicator = vbt.IndicatorFactory(
    class_name = "CostumIndicator",
    short_name = "Custom",
    input_names = ['close'],
    param_names = ['rsi_window', 'entry', 'exit'],
    output_names = ['signal']
    ).from_apply_func(custom_indicator, rsi_window=14, entry=30, exit=70)



# get the result
rsi_window=np.arange(10, 40, step=2, dtype=int)

master_returns = []

for window in rsi_window:
  result = my_indicator.run(btc_price,
                            rsi_window=window,
                            entry=np.arange(10, 40, step=4, dtype=float),
                            exit=np.arange(60, 85, step=4, dtype=float),
                            param_product=True
                            )
  # print(result.signal.to_string())

  # get the entris and exits
  entris = result.signal == 1.0
  exits = result.signal == -1.0

  # run the portfolio
  pf = vbt.Portfolio.from_signals(btc_price, entris, exits, freq='1m')
  # print(pf.stats())
  # returns = pf.total_return()
  master_returns.append(pf.total_return())

print(master_returns)
# If you want to optimize each pair
# returns = returns[returns.index.isin(['BTC-USD'], level='symbol')]

# returns = returns.groupby(level=['Custom_exit', 'Custom_entry', 'symbol']).mean()

returns = pd.concat(master_returns)
print(returns.to_string())

print(returns.max())
print(returns.idxmax())

# If you want to plot (only for one pair)
# pf.plot().show()

## Avoid overfitting

In [8]:
now = datetime.datetime.now()
before = now - datetime.timedelta(days=3)

btc_price = vbt.YFData.download(
    "BTC-USD",
    missing_index='drop',
    interval='1m',
    start=before.timestamp(),
    end=now.timestamp()).get("Close")

btc_price, range_indexes = btc_price.vbt.range_split(n=100, range_len=1440)

def optimize_rsi(close, window, entry, exit):
  rsi = vbt.RSI.run(close, window=window).rsi
  return rsi < entry, rsi > exit

rsi_ind = vbt.IndicatorFactory(
    class_name='optimizeRSI',
    short_name='rsi',
    input_names=['close'],
    param_names=['window', 'entry', 'exit'],
    output_names=['entries', 'exits']
    ).from_apply_func(
        optimize_rsi,
        window=14,
        entry=30,
        exit=70)

step_size = 10
entries = np.arange(10, 45, step=step_size, dtype=int)
exits = np.arange(55, 95, step=step_size, dtype=int)
windows = np.arange(10, 45, step=step_size, dtype=int)

rsi_res = rsi_ind.run(
    btc_price,
    window=windows,
    entry=entries,
    exit=exits,
    param_product=True
    )

rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits

rsi_exits.iloc[-1, :] = True

rsi_pf =vbt.Portfolio.from_signals(btc_price, rsi_entries, rsi_exits, freq="1T", fees=0.001)

fig = rsi_pf.total_return().vbt.volume(
    x_level="rsi_exit",
    y_level="rsi_entry",
    z_level="rsi_window",
    slider_level="split_idx")

fig.show()

In [12]:
rsi_tot_returns = rsi_pf.total_return().groupby(
    level=["rsi_exit", "rsi_entry"]).mean()

fig = rsi_tot_returns.vbt.heatmap(
    x_level="rsi_exit",
    y_level="rsi_entry")

fig.show()

print(rsi_tot_returns)

rsi_exit  rsi_entry
55        10          -0.006423
          20          -0.011884
          30          -0.021518
          40          -0.031971
65        10          -0.005316
          20          -0.009464
          30          -0.012307
          40          -0.011894
75        10          -0.003976
          20          -0.005857
          30          -0.001076
          40           0.005027
85        10           0.000064
          20           0.003793
          30           0.018760
          40           0.027339
Name: total_return, dtype: float64


In [13]:
box = vbt.plotting.Box(
    data=rsi_tot_returns,
    trace_names=["RSI_strat"])

box.fig.show()

In [16]:
def random_signal(close):
  return np.random.randint(0, 2, close.shape)

rand_ind = vbt.IndicatorFactory(
    class_name='Random',
    short_name='rand',
    input_names=['close'],
    output_names=['signal']
    ).from_apply_func(
        random_signal)

rand_res = rand_ind.run(btc_price)

rand_entries = rand_res.signal == 1
rand_exits= rand_res.signal == 0
rand_exits.iloc[-1, :] = True

rand_pf =vbt.Portfolio.from_signals(btc_price, rand_entries, rand_exits, freq="1T")

rand_tot_returns = rand_pf.total_return()

box = vbt.plotting.Box(
    data=rand_tot_returns,
    trace_names=["random"])

box.fig.show()

In [21]:
rsi_res = rsi_ind.run(
    btc_price,
    window=14,
    entry=35,
    exit=85)

rsi_entries = rsi_res.entries
rsi_exits = rsi_res.exits

rsi_exits.iloc[-1, :] = True

rsi_pf =vbt.Portfolio.from_signals(btc_price, rsi_entries, rsi_exits, freq="1T", fees=0.001)

rsi_tot_returns = rsi_pf.total_return()
print(len(list(rsi_tot_returns)))

100


In [22]:


df = pd.DataFrame({
    "rsi":list(rsi_tot_returns),
    "rand":list(rand_tot_returns)})

box = vbt.plotting.Box(
    data=df,
    trace_names=["rsi", "random"])

box.fig.show()